# DATA20001 Deep Learning - Group Project
## Image project

21/05/2020
- Second test of testing [resnext50_32x4d](https://pytorch.org/docs/stable/torchvision/models.html#id27) pretrained transfer learning model with hidden fully-connected layer with 256 nodes.
  - this time with more epochs and fixed model selection criteria (f1)
- Data preprocessing includes: (same preprocessing steps as exp_transfer_learning_2.ipynb)
  - shuffle
  - split: train=17k, val=1.5k, test=1.5k
  - data augmentation


In [1]:
# automatically reload dependencies and repository content so that kernel need not be restarted
%load_ext autoreload
%autoreload 2

## Import dependencies and select correct device

In [2]:
# import dependencies
import train_model
import eval_model
import utils
from image_dataset import ImageDataset
from data_augmentation import DataAugmentation

import time
import torch
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [3]:
device = None
if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda:0")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using CPU


## Define datasets

- shuffle
- train: 17k
- valid: 1.5k
- test: 1.5k

In [4]:
mapping_file = "file_to_labels_table.csv"
df = pd.read_csv(mapping_file)
train_idx, val_idx = utils.get_train_val_indexes(df, 0.85, shuffle=True)
val_test_split_idx = int(val_idx.shape[0]*.5)
test_idx = val_idx[:val_test_split_idx]
val_idx = val_idx[val_test_split_idx:]

dataAugmentation = DataAugmentation()
trainDataset = ImageDataset(train_idx, dataAugmentation=dataAugmentation)
valDataset = ImageDataset(val_idx)
testDataset = ImageDataset(test_idx)

print("Saving test dataset temporarily so test can be performed after restarting kernel.")
utils.save_pickle(testDataset, "tmp_test_data_exp_transfer_learning_4.pkl")
print("# Train:", len(trainDataset))
print("# Valid:", len(valDataset))
print("# Test:", len(testDataset))

Saving test dataset temporarily so test can be performed after restarting kernel.
# Train: 17000
# Valid: 1500
# Test: 1500


## Define the model architecture

The initial transfer learning implementation

In [5]:
preTrainedModel = models.resnext50_32x4d(pretrained=True)

for param in preTrainedModel.parameters():
    param.requires_grad = False

num_ftrs = preTrainedModel.fc.in_features
num_classes = 14

preTrainedModel.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, num_classes),
    torch.nn.Sigmoid()
)

model = preTrainedModel.to(device)

## Define test case

Hyperparam search could be defined here. No hyperparam searches in this notebook as purpose of this test is to test the effect of data preprocessing technique on initial transfer learning model architecture and see how longer run on epochs affect loss.

**Test objective**:

- Testing second implementation of [resnext50_32x4d](https://pytorch.org/docs/stable/torchvision/models.html#id27) implementation of pretrained transfer learning models
  - New addition: hidden fully-connected layer with 256 nodes
- Data preprocessing includes data augmentation, shuffle and split train=17k, val=1.5k, test=1.5k
- Data augmentation IS applied

**Training results**:



In [7]:
n_epochs = 10

model, logs = train_model.train_model(
    model,
    trainDataset,
    valDataset,
    device,
    numberOfEpochs=n_epochs,
    returnLogs=True
)

Epoch 0/10
----------
Validating...

Progress: 100%

Loss: 0.6826 Accuracy: 0.5869
Precision: 0.0736 Recall: 0.4142
F1: 0.1251 Duration: 1m 12s

Epoch 1/10
----------
Training...

Progress: 100%

Loss: 0.1553 Accuracy: 0.9434
Precision: 0.6888 Recall: 0.4012
F1: 0.5070 Duration: 20m 0s

Validating...

Progress: 100%

Loss: 0.1362 Accuracy: 0.9470
Precision: 0.6002 Recall: 0.7702
F1: 0.6747 Duration: 1m 13s

Epoch 2/10
----------
Training...

Progress: 100%

Loss: 0.1371 Accuracy: 0.9489
Precision: 0.7245 Recall: 0.4767
F1: 0.5751 Duration: 19m 34s

Validating...

Progress: 100%

Loss: 0.1197 Accuracy: 0.9567
Precision: 0.8316 Recall: 0.4916
F1: 0.6180 Duration: 1m 14s

Epoch 3/10
----------
Training...

Progress: 100%

Loss: 0.1322 Accuracy: 0.9510
Precision: 0.7433 Recall: 0.4960
F1: 0.5950 Duration: 19m 59s

Validating...

Progress: 100%

Loss: 0.1095 Accuracy: 0.9591
Precision: 0.7099 Recall: 0.7208
F1: 0.7153 Duration: 1m 14s

Epoch 4/10
----------
Training...

Progress: 100%

Loss

## Save model and logs

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [8]:
pd.DataFrame(logs["val"]).to_csv("exp_transfer_learning_4_2_val_logs.csv", index=False)
pd.DataFrame(logs["train"]).to_csv("exp_transfer_learning_4_2_train_logs.csv", index=False)
torch.save(model, 'exp_transfer_learning_4_2_model.pkl')

## Test model

- Load model back from state_dict and perform test on testing set
- Load test-set
- Load training time logs of performance

In [7]:
model = torch.load("exp_transfer_learning_4_2_model.pkl")
testDataset = utils.load_pickle("tmp_test_data_exp_transfer_learning_4.pkl")
val_logs = pd.read_csv("exp_transfer_learning_4_2_val_logs.csv")
train_logs = pd.read_csv("exp_transfer_learning_4_2_train_logs.csv")
y_hats, y_trues = eval_model.test_model(model, testDataset, device)

### Validation set performance during training

In [10]:
logs

{'epoch': [0, 1, 2, 3, 4],
 'accuracy': [tensor(0.9579, dtype=torch.float64),
  tensor(0.9565, dtype=torch.float64),
  tensor(0.9581, dtype=torch.float64),
  tensor(0.9597, dtype=torch.float64),
  tensor(0.9601, dtype=torch.float64)],
 'loss': [0.1101102386471771,
  0.11380755911554609,
  0.10892651088748659,
  0.10757335066511517,
  0.10641117515166601]}

### Testing set performance

In [8]:
print("Testing set results.")
print(f" Accuracy\t{round(eval_model.get_metric(y_trues, y_hats, 'accuracy'), 3)}")
print(f" Precision\t{round(eval_model.get_metric(y_trues, y_hats, 'precision'), 3)}")
print(f" Recall \t{round(eval_model.get_metric(y_trues, y_hats, 'recall'), 3)}")
print(f" F1-score\t{round(eval_model.get_metric(y_trues, y_hats, 'f1'), 3)}")

Testing set results.
 Accuracy	0.959
 Precision	0.745
 Recall 	0.688
 F1-score	0.715


### Extra analysis

How many positives and negatives has been predicted.

Tuples like: **(label, count)**

In [8]:
list(zip(*np.unique(y_hats, return_counts=True)))

[(0, 19592), (1, 1408)]

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')